<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Finetuning/blob/main/60-STREANIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab + Streamlit 데이터 시각화 및 AI 예측 인터페이스 실습 교안

## 1. 개요

- **목표:** Colab 환경에서 Streamlit을 사용해 데이터 분석, 시각화, AI 모델 예측까지 경험해보기
- **주요 내용:**  
    - Colab에서 Streamlit 앱 실행 및 외부 접속  
    - 데이터 업로드, EDA, 시각화  
    - 간단한 AI 예측(선형회귀) 인터페이스 구현  
    - ngrok을 이용한 외부 URL 접속

---

## 2. 실습 준비

### 2-1. Colab 새 노트북 준비

- [Google Colab](https://colab.research.google.com/)에서 새 노트북을 엽니다.

---

## 3. Streamlit & ngrok 기반 앱 실행 코드

아래 **전체 코드를 하나의 셀에 복사-실행**합니다.

```python
# 1. 라이브러리 설치
!pip install streamlit==1.32.2 pyngrok pandas scikit-learn matplotlib --quiet

# 2. Colab 파일 업로드
import os
from google.colab import files

uploaded = files.upload()
if uploaded:
    filename = next(iter(uploaded))

# 3. ngrok 설정
from pyngrok import ngrok
ngrok.kill() # 기존 터널 종료

# 4. app.py 코드 작성
app_code = '''
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import os

st.set_page_config(page_title="데이터 시각화 및 AI 예측", layout="wide")
st.title("🧊 Streamlit 데이터 시각화 & AI 예측 데모")

# Colab 업로드 파일 검색
uploaded_files = [f for f in os.listdir() if f.endswith('.csv')]
if uploaded_files:
    st.sidebar.header("1️⃣ 데이터 파일 선택")
    filename = st.sidebar.selectbox("분석할 파일을 선택하세요", uploaded_files)
    df = pd.read_csv(filename)
    st.subheader("미리보기")
    st.dataframe(df.head())

    st.sidebar.header("2️⃣ EDA & 시각화")
    st.subheader("기초 통계")
    st.write(df.describe())

    st.subheader("컬럼별 분포 (히스토그램)")
    col_option = st.selectbox("히스토그램을 볼 컬럼을 선택하세요", df.columns)
    fig, ax = plt.subplots()
    ax.hist(df[col_option].dropna(), bins=30)
    st.pyplot(fig)

    st.sidebar.header("3️⃣ AI 예측")
    target_col = st.sidebar.selectbox("예측할 타겟(종속) 컬럼을 선택하세요", df.columns)
    feature_cols = st.sidebar.multiselect(
        "특징(독립) 컬럼을 선택하세요",
        [col for col in df.columns if col != target_col]
    )

    if len(feature_cols) > 0 and target_col:
        X = df[feature_cols]
        y = df[target_col]
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

        model = LinearRegression()
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        st.write(f"**예측 모델(R²): {score:.2f}**")

        st.markdown("#### 입력값을 넣어 예측해 보세요")
        input_data = []
        for col in feature_cols:
            val = st.number_input(f"{col} 입력", float(df[col].min()), float(df[col].max()), float(df[col].mean()))
            input_data.append(val)
        if st.button("예측하기"):
            pred = model.predict([input_data])
            st.success(f"예측 결과: {pred[0]:.2f}")
else:
    st.info("Colab에 .csv 파일을 업로드한 뒤 새로고침하세요.")
'''

with open('app.py', 'w', encoding='utf-8') as f:
    f.write(app_code)

# 5. ngrok으로 Streamlit 서버 노출
public_url = ngrok.connect(8501)
print(f'▶️ 아래 링크를 클릭하면 Colab에서 실행 중인 Streamlit 앱에 접속할 수 있습니다:\n{public_url}')

# 6. Streamlit 실행 (백그라운드)
!streamlit run app.py &>/content/log.txt &
